In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import Image
from numpy import asarray
import glob
import cv2
import tensorflow as tf
from tensorflow import keras

Import Model

In [2]:
UnWeighted_Model=keras.models.load_model('drive/MyDrive/CNN_91.57Acc_87.22')

Import Data

In [3]:
X_train=np.load('drive/MyDrive/X_train_strat.npy')

In [4]:
y_train=np.load('drive/MyDrive/y_train_strat.npy')

In [5]:
X_val=np.load('drive/MyDrive/X_val_strat.npy')

In [6]:
y_val=np.load('drive/MyDrive/y_val_strat.npy')

Get val data probabilities

In [7]:
y_proba=UnWeighted_Model.predict(X_val)

In [8]:
y_proba[1]

array([1.8700127e-01, 2.0648569e-03, 6.5014623e-02, 5.6939271e-06,
       2.4076574e-09, 3.1269633e-03, 8.9075873e-03, 5.9864538e-05,
       9.2869840e-04, 3.2675439e-03, 1.4010909e-21, 2.3089875e-03,
       5.6240433e-01, 1.2087727e-01, 2.8029918e-03, 1.9908721e-04,
       5.3067452e-05, 1.5319049e-02, 8.9288881e-07, 2.1978799e-04,
       1.3941696e-11, 1.0983007e-07, 3.4774159e-10, 9.2288531e-07,
       2.2313192e-02, 1.0492831e-03, 3.0943474e-06, 2.0708006e-03],
      dtype=float32)

In [9]:
# Turn one hot array into integers
y_val_1d=np.argmax(y_val, axis=1)
y_predictions=np.argmax(y_proba, axis=1)

In [10]:
y_dataframe=pd.DataFrame({'y_predictions': y_predictions,
                          'y_actual': y_val_1d
                          })

In [11]:
y_dataframe

,y_predictions,y_actual
0,25,25
1,12,12
2,8,2
3,23,23
4,13,14
...,...,...
8268,1,1
8269,13,13
8270,1,1
8271,1,1


In [12]:
y_proba_list=[]

for each in y_proba:
  y_proba_list.append(each)

In [13]:
#y_dataframe['y_proba']=y_proba_list

In [14]:
#round to 5 decimals

rounded_proba_list=[]
for each_array in y_proba_list:
  new_array=[]
  for each_number in each_array:
    rounded_number=round(each_number, ndigits=5)
    new_array.append('{0:6f}'.format(rounded_number))
  rounded_proba_list.append(new_array)

In [15]:
y_dataframe['y_rounded_proba']=rounded_proba_list

In [16]:
y_dataframe.head(5)

,y_predictions,y_actual,y_rounded_proba
0,25,25,"[0.000290, 0.000050, 0.444450, 0.000000, 0.000..."
1,12,12,"[0.187000, 0.002060, 0.065010, 0.000010, 0.000..."
2,8,2,"[0.000000, 0.000050, 0.368020, 0.000210, 0.000..."
3,23,23,"[0.000000, 0.000000, 0.000000, 0.000000, 0.000..."
4,13,14,"[0.001320, 0.002030, 0.005430, 0.000550, 0.000..."


In [17]:
## locate incorrectly classified animals
incorrect_df=y_dataframe.loc[(y_dataframe['y_predictions'])!=(y_dataframe['y_actual'])]

In [18]:
incorrect_df

,y_predictions,y_actual,y_rounded_proba
2,8,2,"[0.000000, 0.000050, 0.368020, 0.000210, 0.000..."
4,13,14,"[0.001320, 0.002030, 0.005430, 0.000550, 0.000..."
6,12,13,"[0.089220, 0.001030, 0.009610, 0.000010, 0.000..."
8,13,12,"[0.195130, 0.004920, 0.005880, 0.000130, 0.000..."
15,2,16,"[0.000010, 0.000000, 0.990060, 0.000000, 0.000..."
...,...,...,...
8210,2,8,"[0.000000, 0.000010, 0.678840, 0.000000, 0.000..."
8227,8,2,"[0.000010, 0.003020, 0.128240, 0.004250, 0.000..."
8233,2,26,"[0.000000, 0.000010, 0.464560, 0.000600, 0.000..."
8234,8,16,"[0.000000, 0.000000, 0.018780, 0.000000, 0.000..."


In [19]:
## add columns for predicted probability and predicted probability of actual class
predicted_class_probability=[]
actual_class_probability=[]
for index, row in incorrect_df.iterrows():
  p=row['y_predictions']
  a=row['y_actual']
  pred_proba=row['y_rounded_proba'][p]
  act_proba=row['y_rounded_proba'][a]
  predicted_class_probability.append(pred_proba)
  actual_class_probability.append(act_proba)

In [20]:
incorrect_df['predicted_class_probability']=predicted_class_probability
incorrect_df['actual_class_probability']=actual_class_probability
incorrect_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,y_predictions,y_actual,y_rounded_proba,predicted_class_probability,actual_class_probability
2,8,2,"[0.000000, 0.000050, 0.368020, 0.000210, 0.000...",0.595070,0.368020
4,13,14,"[0.001320, 0.002030, 0.005430, 0.000550, 0.000...",0.456610,0.191300
6,12,13,"[0.089220, 0.001030, 0.009610, 0.000010, 0.000...",0.582400,0.215040
8,13,12,"[0.195130, 0.004920, 0.005880, 0.000130, 0.000...",0.394860,0.042490
15,2,16,"[0.000010, 0.000000, 0.990060, 0.000000, 0.000...",0.990060,0.004100
...,...,...,...,...,...
8210,2,8,"[0.000000, 0.000010, 0.678840, 0.000000, 0.000...",0.678840,0.008080
8227,8,2,"[0.000010, 0.003020, 0.128240, 0.004250, 0.000...",0.412930,0.128240
8233,2,26,"[0.000000, 0.000010, 0.464560, 0.000600, 0.000...",0.464560,0.347810
8234,8,16,"[0.000000, 0.000000, 0.018780, 0.000000, 0.000...",0.580400,0.376710


In [21]:
#find differences in predicted vs actual probability
incorrect_df['difference']=(incorrect_df['predicted_class_probability'].astype(float))-(incorrect_df['actual_class_probability'].astype(float))
incorrect_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,y_predictions,y_actual,y_rounded_proba,predicted_class_probability,actual_class_probability,difference
2,8,2,"[0.000000, 0.000050, 0.368020, 0.000210, 0.000...",0.595070,0.368020,0.22705
4,13,14,"[0.001320, 0.002030, 0.005430, 0.000550, 0.000...",0.456610,0.191300,0.26531
6,12,13,"[0.089220, 0.001030, 0.009610, 0.000010, 0.000...",0.582400,0.215040,0.36736
8,13,12,"[0.195130, 0.004920, 0.005880, 0.000130, 0.000...",0.394860,0.042490,0.35237
15,2,16,"[0.000010, 0.000000, 0.990060, 0.000000, 0.000...",0.990060,0.004100,0.98596
...,...,...,...,...,...,...
8210,2,8,"[0.000000, 0.000010, 0.678840, 0.000000, 0.000...",0.678840,0.008080,0.67076
8227,8,2,"[0.000010, 0.003020, 0.128240, 0.004250, 0.000...",0.412930,0.128240,0.28469
8233,2,26,"[0.000000, 0.000010, 0.464560, 0.000600, 0.000...",0.464560,0.347810,0.11675
8234,8,16,"[0.000000, 0.000000, 0.018780, 0.000000, 0.000...",0.580400,0.376710,0.20369


In [22]:
# ## get counts for each misclassified class
missed_counts=pd.DataFrame(incorrect_df['y_actual'].value_counts())
missed_counts.reset_index(inplace=True)

In [23]:
y_train_counts=pd.DataFrame(y_dataframe['y_actual'].value_counts())
y_train_counts.reset_index(inplace=True)

In [24]:
y_reference=pd.merge(y_train_counts,missed_counts, on='index')
y_reference.set_index('index', inplace=True)
y_reference.rename(columns={'y_actual_x':'y_in_training', 'y_actual_y':'missed_ys'})

,y_in_training,missed_ys
index,,
2,1740,131
0,1207,29
13,1199,133
1,783,40
8,547,146
7,512,10
14,392,43
16,276,62
25,274,58


In [25]:
table25=incorrect_df.loc[incorrect_df['y_actual']==25]

In [26]:
table25

,y_predictions,y_actual,y_rounded_proba,predicted_class_probability,actual_class_probability,difference
69,2,25,"[0.000580, 0.002000, 0.868230, 0.000500, 0.000...",0.868230,0.016290,0.85194
246,8,25,"[0.000090, 0.000030, 0.002220, 0.000000, 0.049...",0.839110,0.006830,0.83228
393,0,25,"[0.999990, 0.000000, 0.000000, 0.000000, 0.000...",0.999990,0.000000,0.99999
442,2,25,"[0.000050, 0.001920, 0.268810, 0.000880, 0.000...",0.268810,0.221820,0.04699
566,16,25,"[0.000000, 0.000320, 0.105850, 0.000370, 0.000...",0.407540,0.214060,0.19348
852,8,25,"[0.000000, 0.000000, 0.015780, 0.000000, 0.002...",0.853130,0.008750,0.84438
934,2,25,"[0.000000, 0.000000, 0.709270, 0.000000, 0.000...",0.709270,0.257410,0.45186
1045,8,25,"[0.000000, 0.000250, 0.407970, 0.000190, 0.000...",0.451580,0.004480,0.44710
1143,7,25,"[0.000000, 0.000000, 0.001530, 0.000000, 0.000...",0.988140,0.006510,0.98163
1207,2,25,"[0.000010, 0.000000, 0.603700, 0.000000, 0.000...",0.603700,0.370210,0.23349


In [27]:
## find average difference
avg=(table25['difference'].sum())/len(table25['difference'])
avg

##  8: 0.42
## 16: 0.49
## 25: 0.43

0.43305034482758625

In [28]:
## find median difference
import statistics
med=statistics.median(table25['difference'])
med

##  8: 0.40
## 16: 0.47
## 25: 0.35

0.350825

Create Weights for underepresented classes

In [29]:
## lots of classes get incorrectly predicted as 2's

#Manually adjusting weights for only the poorly classified classes
adjusted_class_weights={
    0: 1.,
    1: 1.,
    2: 1.,
    3: 1.,
    4: 1.,
    5: 1.1,
    6: 1.,
    7: 1.,
    8: 1.15,
    9: 1.,
    10: 1.,
    11: 1.,
    12: 1.,
    13: 1.,
    14: 1.,
    15: 1.,
    16: 1.15,
    17: 1.,
    18: 1.,
    19: 1.,
    20: 1.,
    21: 1.,
    22: 1.,
    23: 1.,
    24: 1.,
    25: 1.3,
    26: 1.,
    27: 1.}

Add weights to training

In [30]:
from tensorflow.keras import layers
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

model4 = keras.models.Sequential()
model4.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(150,200,1),padding='same'))
model4.add(MaxPooling2D((2, 2),padding='same'))
model4.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
model4.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model4.add(Dropout(0.28))
model4.add(Conv2D(128, (3, 3), activation='relu',padding='same'))                 
model4.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model4.add(Dropout(0.35))
model4.add(Conv2D(256, (3, 3), activation='relu',padding='same'))                 
model4.add(MaxPooling2D(pool_size=(3, 3),padding='same'))
model4.add(Dropout(0.40))
model4.add(Conv2D(512, (3, 3), activation='relu',padding='same'))          
model4.add(MaxPooling2D(pool_size=(5, 5),padding='same'))               ## changed from 4,4 to 5,5
model4.add(Dropout(0.40))
model4.add(Conv2D(512, (3, 3), activation='relu',padding='same'))        ## 256 to 512 and dropout to .50       
model4.add(MaxPooling2D(pool_size=(8,8),padding='same'))                 ##changed from 6,6 to 8,8
model4.add(Dropout(0.50))
model4.add(Conv2D(512, (3, 3), activation='relu',padding='same'))        ##       
model4.add(MaxPooling2D(pool_size=(6,6),padding='same'))                 ##  new layer
model4.add(Dropout(0.50))                                                ##
model4.add(Conv2D(256, (3, 3), activation='relu',padding='same'))      ## added conv, pooling layer           
model4.add(MaxPooling2D(pool_size=(3, 3),padding='same'))
model4.add(Dropout(0.35))
model4.add(Flatten())
model4.add(Dense(300, activation='relu'))               
model4.add(Dense(28, activation='softmax'))

In [31]:

model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [32]:
# callback to save best parameters
checkpoint=tf.keras.callbacks.ModelCheckpoint(
    filepath='drive/MyDrive/CNN4_2',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

In [33]:
model4.fit(X_train, y_train, batch_size=110, epochs=100, validation_data=(X_val, y_val), callbacks=[checkpoint])

Epoch 1/100
301/301 [==============================] - 58s 171ms/step - loss: 2.4127 - accuracy: 0.2589 - val_loss: 1.8351 - val_accuracy: 0.4352
Epoch 2/100
301/301 [==============================] - 46s 153ms/step - loss: 1.6485 - accuracy: 0.5064 - val_loss: 1.3832 - val_accuracy: 0.5850
Epoch 3/100
301/301 [==============================] - 47s 155ms/step - loss: 1.3843 - accuracy: 0.5892 - val_loss: 1.2811 - val_accuracy: 0.6055
Epoch 4/100
301/301 [==============================] - 47s 155ms/step - loss: 1.2740 - accuracy: 0.6199 - val_loss: 1.1135 - val_accuracy: 0.6608
Epoch 5/100
301/301 [==============================] - 47s 156ms/step - loss: 1.1887 - accuracy: 0.6400 - val_loss: 1.0583 - val_accuracy: 0.6719
Epoch 6/100
301/301 [==============================] - 47s 156ms/step - loss: 1.1134 - accuracy: 0.6633 - val_loss: 1.0017 - val_accuracy: 0.7019
Epoch 7/100
301/301 [==============================] - 47s 156ms/step - loss: 1.0475 - accuracy: 0.6838 - val_loss: 0.8832 -

In [34]:
## weighted all 1's                            Accuracy: 79   Val Accuracy: 79  after 29 epochs
## No weights                                  Accuracy: 79   Val Accuracy: 78  after 37 epochs
## No weights batch size 80                    Accuracy: 85   Val Accuracy: 81  after 30 epochs
## No weights batch size 90                    Accuracy: 88   Val Accuracy: 83.9  after 46 epochs?
## NW, BS 90, last conv2d 256                  Accuracy: 84   Val Accuracy: 83  after 26 epochs
## NW, bs 90, last conv2d 256, dense 400       Accuracy: 88   Val Accuracy: 85.29 after 53 epochs
## NW, bs 90, ', dense 300, 512 pool(4x4)      Accuracy: 86.9 Val Accuracy: 85.75 after 48 epochs

In [35]:
UnWeighted_Model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 200, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 100, 32)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 100, 64)       18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 38, 50, 64)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 38, 50, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 38, 50, 128)       7